## Qusetion 1

a). Please fill in the missing code to train a 3-layer ConvNet on the CIFAR10 dataset.

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 32

train_transform =transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])


trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=train_transform)

validation_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=True)

# for (x_train, y_train) in train_loader:
#     print (x_train.size())
#     break

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, 1, 1)
        
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        
        self.bn3 = nn.BatchNorm2d(128)
        
        self.fc1 = nn.Linear(128*32*32, 10)


    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        
        ## forward x to relu activation function.
        x = F.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        
        ## forward x to relu activation function.
        x = F.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        
        ## forward x to relu activation function.
        x = F.relu(x)
        
        ## flatten operation
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        
        ## forward x to softmax activation function.
        output = F.softmax(x, dim=1)
        
        return output
    
def train(epoch,log_interval=100):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        
        ## zero gradient buffers
        optimizer.zero_grad()
        
        output = model(data)

        loss = criterion(output, target)

        ## backpropagate
        loss.backward()
        
        ## update weights
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        
        pred = output.data.max(1)[1]
        
        correct += pred.eq(target.data).sum()

    val_loss /= len(validation_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct.to(torch.float32) / len(validation_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))


model = Net().to(device)

## define the SGD optimizer with lr=0.01 and momentum=0.5
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

## define the crosstntropy loss.
criterion = nn.CrossEntropyLoss()

epochs = 2
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(epoch)
    validate(lossv, accv)


Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.308548
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.331400
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.235070
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2.210311
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.159349
Train Epoch: 1 [16000/50000 (32%)]	Loss: 2.258880
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.206248
Train Epoch: 1 [22400/50000 (45%)]	Loss: 2.210301
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.050405
Train Epoch: 1 [28800/50000 (58%)]	Loss: 2.194498
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.289950
Train Epoch: 1 [35200/50000 (70%)]	Loss: 2.015035
Train Epoch: 1 [38400/50000 (77%)]	Loss: 2.211916
Train Epoch: 1 [41600/50000 (83%)]	Loss: 2.141346
Train Epoch: 1 [44800/50000 (90%)]	Loss: 2.209134
Train Epoch: 1 [48000/50000 (96%)]	Loss: 1.888746

Validation set: Average loss: 2.1753, Accuracy: 2812/10000 (28.12%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 2.144143
Train Epoch: 2 [32

b). Please apply knowledge distillation by using ResNet18 as a teacher network to improve the accuracy of the above 3-layer ConvNet, which can be treated as a student network. Please fill in the missing code.

In [17]:
import os
import time
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import resnet


## load the saved model for ResNet-18 so you do not to train ResNet-18
def load_checkpoint(checkpoint, model):

    if not os.path.exists(checkpoint):
        raise("File doesn't exist {}".format(checkpoint))

    checkpoint = torch.load(checkpoint, map_location=lambda storage, loc: storage)
    model.load_state_dict(checkpoint['state_dict'])

    return checkpoint

def cal_loss_kd(T, alpha, output_student, output_teacher):
    """
    Compute the knowledge-distillation (KD) loss given outputs, labels.
    "Hyperparameters": temperature and alpha
    
    Hint: since the built-in CrossEntropy loss in pytorch only takes “(output, target)” 
    where the target is the class index, which is different from knowledge distillation loss,
    you can consider KL Divergence in pytorch. Please find out the definition of CrossEntropy,
    KL Divergence, their difference and usage in pytorch to carefully complete this part.

    """
    criterion_kd = nn.KLDivLoss()
    
    ## calculate the soft distribution of the student output with temperature T
    student_soft = F.log_softmax(output_student / T, dim=1)
    
    ## calculate the soft distribution of the teacher output with temperature T
    teacher_soft = F.softmax(output_teacher / T, dim=1)
    
    KD_loss = criterion_kd(student_soft,teacher_soft)* (alpha * T * T)
    
    return KD_loss


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

batch_size = 32

train_transform =transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=train_transform)

validation_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=True)


teacher_model = resnet.ResNet18().to(device)
teacher_checkpoint = 'best.pth.tar'
load_checkpoint(teacher_checkpoint, teacher_model)
teacher_model.eval()

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, 1, 1)
        
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        
        self.bn3 = nn.BatchNorm2d(128)
        
        self.fc1 = nn.Linear(128*32*32, 10)


    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        
        ## forward x to relu activation function.
        x = F.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        
        ## forward x to relu activation function.
        x = F.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        
        ## forward x to relu activation function.
        x = F.relu(x)
        
        ## flatten operation
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        
        ## forward x to softmax activation function.
        output = F.softmax(x, dim=1)
        
        return output


def train(epoch, log_interval=100):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        ## zero gradient buffers
        optimizer.zero_grad()

        ## forward data through the student model
        output_student = model(data)

        ## forward data through the teacher model
        output_teacher = teacher_model(data)

        ## set the output of the teacher attribute .requires_grad as False
        output_teacher = Variable(output_teacher, requires_grad=False)

        T=5
        alpha = 0.5

        loss_kd = cal_loss_kd(T, alpha, output_student, output_teacher)

        loss_class = criterion(output_student, target)*(1-alpha)

        loss = loss_kd + loss_class

        ## backpropagate
        loss.backward()

        ## update weights
        optimizer.step()

        if (batch_idx+1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss_kd: {:.6f}\tLoss_class: {:.6f}\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss_kd.data.item(),loss_class.data.item(),loss.data.item()),flush=True)


def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()

        pred = output.data.max(1)[1]

        correct += pred.eq(target.data).sum()

    val_loss /= len(validation_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct.to(torch.float32) / len(validation_loader.dataset)
    accuracy_vector.append(accuracy)

    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))


model = Net().to(device)

## define the SGD optimizer with lr=0.01 and momentum=0.5
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

## define the crosstntropy loss for the student model. Please note that we do not use softmax output in the network.
criterion = nn.CrossEntropyLoss()

epochs = 2
lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(epoch)
    validate(lossv, accv)



Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [3168/50000 (6%)]	Loss_kd: 0.854856	Loss_class: 1.067331	Loss: 1.922187
Train Epoch: 1 [6368/50000 (13%)]	Loss_kd: 0.701941	Loss_class: 1.048110	Loss: 1.750051
Train Epoch: 1 [9568/50000 (19%)]	Loss_kd: 0.785347	Loss_class: 1.106295	Loss: 1.891642
Train Epoch: 1 [12768/50000 (26%)]	Loss_kd: 0.734804	Loss_class: 0.992062	Loss: 1.726865
Train Epoch: 1 [15968/50000 (32%)]	Loss_kd: 0.771739	Loss_class: 1.110948	Loss: 1.882688
Train Epoch: 1 [19168/50000 (38%)]	Loss_kd: 0.699138	Loss_class: 1.024544	Loss: 1.723682
Train Epoch: 1 [22368/50000 (45%)]	Loss_kd: 0.698171	Loss_class: 1.027328	Loss: 1.725499
Train Epoch: 1 [25568/50000 (51%)]	Loss_kd: 0.773947	Loss_class: 1.109962	Loss: 1.883909
Train Epoch: 1 [28768/50000 (58%)]	Loss_kd: 0.766216	Loss_class: 1.086311	Loss: 1.852527
Train Epoch: 1 [31968/50000 (64%)]	Loss_kd: 0.764030	Loss_class: 1.051860	Loss: 1.815889
Train Epoch: 1 [35168/50000 (70%)]	Los